# Computation of transverse-Ising jump operators

In [1]:
include("notebook_preamble.jl");

In [2]:
using BenchmarkTools, Revise
includet("TransverseIsingModels.jl")
using .TransverseIsingModels
TIM = TransverseIsingModels;

In [3]:
title(s::TransverseIsingModel) = latexstring("$(s.N)-spin transverse Ising model with \$\\lambda =\$ $(s.λ)")
title(s::TranslationInvariantTransverseIsingModel) = latexstring("$(s.N)-spin transverse* Ising model with \$\\lambda =\$ $(s.λ)");

In [4]:
# These can likely be made more efficient.
opip(A, B) = tr(dagger(A) * B)
opip(A) = tr(dagger(A) * A)
opnorm(A) = √opip(A)
opnormalize(A) = A / opnorm(A)
opcos(A, B) = real(abs(opip(A, B)) / (opnorm(A) * opnorm(B)))

# project(P, J) = J == zero(J) ? zero(eltype(J)) : opip(P / opip(P), J)
project(P, J) = J == zero(J) ? missing : opip(P / opip(P), J)
# jumpcos(P, J) = J == zero(J) ? real(one(eltype(J))) : opcos(P, J);
jumpcos(P, J) = J == zero(J) ? missing : opcos(P, J);

In [35]:
tim = TransverseIsingModel(5, 0.5)
bes = basiseigen(tim)
As = [changebasis(op(tim, i); basiseigensys=bes)
    for i in 1:tim.N, op in [sx, sy, sz]]
Ps = [changebasis(op(tim, i); basiseigensys=bes)
    for i in 1:tim.N, op in [sx, sy, sz]]
Πs, ωs = jumpinfo(tim; basiseigensys=bes);

In [36]:
using Statistics

In [159]:
function jumpcomputation(s, Πs, ωs, As, Ps)
    M = length(ωs)
    m = sum(values(ωs)) do ΔEs
#         changebasis(op(tim, i); basiseigensys=bes)
        separations = map(0:(s.N-1)) do j
            Cs = map(Iterators.product(As, Ps)) do (A, P)
                J = jumpoperator(ΔEs, A, Πs)
                project(P, J)
            end
            # Translating both sites by i should not change the result.
            testCs = (Cs[i,:,((i+j-1) % s.N) + 1,:] for i in 1:s.N)
            testpairs = zip(testCs, Iterators.rest(testCs, 2))
            reduce((a, b) -> a .& b,
                (TIM.isapprox.(x, y, atol=1e-12) for (x, y) in testpairs))
        end
        reduce((a, b) -> a .& b, separations) |> skipmissing |> all
    end
    m, M
end;

In [161]:
@time jumpcomputation(tim, Πs, ωs, As, Ps)

 10.034344 seconds (11.06 M allocations: 10.885 GiB, 12.27% gc time)


(235, 235)

In [162]:
function jumpcomputation(s, Πs, ωs, bes)
    M = length(ωs)
    m = sum(values(ωs)) do ΔEs
        separations = map(0:(s.N-1)) do j
            # Translating both sites by i should not change the result.
            testCs = map(1:s.N) do i
                map(Iterators.product([sx, sy, sz], [sx, sy, sz])) do (sa, sb)
                    A = changebasis(sa(s, i); basiseigensys=bes)
                    P = changebasis(sb(s, ((i+j-1) % s.N) + 1); basiseigensys=bes)
                    project(P, jumpoperator(ΔEs, A, Πs))
                end
            end
            C0, restCs = Iterators.peel(testCs)
            T = ones(Bool, axes(C0))
            p, _ = reduce(((p, a), b) -> (p .& TIM.isapprox.(a, b, atol=1e-12), b), restCs, init=(T, C0))
            p
        end
        reduce((a, b) -> a .& b, separations) |> skipmissing |> all
    end
    m, M
end;

In [244]:
function J(s, i, j, sa, sb, ΔEs, Πs, bes)
    A = changebasis(sa(s, i); basiseigensys=bes)
    P = changebasis(sb(s, ((i+j-1) % s.N) + 1); basiseigensys=bes)
    project(P, jumpoperator(ΔEs, A, Πs))
end
function jumpcomputation(s)
    bes = basiseigen(s)
    Πs, ωs = jumpinfo(s; basiseigensys=bes)
    sum(values(ωs)) do ΔEs
        all([sx, sy, sz]) do sa
            all([sx, sy, sz]) do sb
                all(0:(s.N-1)) do j
                    (all ∘ skipmissing ∘ map)(2:(s.N-1)) do i
                        TIM.isapprox.(J(s, 1, j, sa, sb, ΔEs, Πs, bes), J(s, i, j, sa, sb, ΔEs, Πs, bes), atol=1e-12)
                    end
                end
            end
        end
    end
end;

In [245]:
@code_warntype jumpcomputation(TransverseIsingModel(5, 0.5))

Variables
  #self#::Core.Compiler.Const(jumpcomputation, false)
  s::TransverseIsingModel
  @_3::Int64
  #847::var"#847#852"{TransverseIsingModel,_A,_B} where _B where _A
  bes::NamedTuple{(:vals, :kets, :P, :Pinv),_A} where _A<:Tuple
  Πs::Dict{_A,_B} where _B where _A
  ωs::Dict{_A,_B} where _B where _A

Body::Any
1 ─       (bes = Main.basiseigen(s))
│   %2  = (:basiseigensys,)::Core.Compiler.Const((:basiseigensys,), false)
│   %3  = Core.apply_type(Core.NamedTuple, %2)::Core.Compiler.Const(NamedTuple{(:basiseigensys,),T} where T<:Tuple, false)
│   %4  = Core.tuple(bes)::Tuple{NamedTuple{(:vals, :kets, :P, :Pinv),_A} where _A<:Tuple}
│   %5  = (%3)(%4)::NamedTuple{(:basiseigensys,),_A} where _A<:Tuple
│   %6  = Core.kwfunc(Main.jumpinfo)::Core.Compiler.Const(Main.TransverseIsingModels.var"#jumpinfo##kw"(), false)
│   %7  = (%6)(%5, Main.jumpinfo, s)::Tuple{Dict{_A,_B} where _B where _A,Dict{_A,_B} where _B where _A}
│   %8  = Base.indexed_iterate(%7, 1)::Core.Compiler.PartialStruct(T

**TODO**
- Just have things take a `basiseigensys` keyword argument.
- Fix everything being wrong.
- Fix slowness (precompilation and type-instability).
- Check with the $\opr{\eta}_k$?

In [ ]:
function jumpcomputation(s, As, Ps; basiseigensys=basiseigensys)
    jumpoperators(tim; basiseigensys=basiseigensys) do jumpop
        Js = (jumpop(A) for A in As)
    #     [jumpcos(P, J) for J in Js, P in Ps]
        [project(P, J) for J in Js, P in Ps]
    end
end

In [ ]:
@time Jωs = jumpcomputation(tim, As, Ps, basiseigensys=ebs);

In [ ]:
# Jω = collect(Jωs)[22][2];
Jω = first(Jωs)[2]
[Jω[1,:,1,:], Jω[2,:,2,:]] .|> display;

In [ ]:
sum(values(Jωs)) do Jω
    reduce((a, b) -> a .& b,
        map(0:(tim.N-1)) do j
            # Translating both sites by i should not change the result.
            testJs = (Jω[i,:,((i+j-1) % tim.N) + 1,:] for i in 1:tim.N)
            testpairs = zip(testJs, Iterators.rest(testJs, 2))
            reduce((a, b) -> a .& b,
                (TIM.isapprox.(x, y, atol=1e-12) for (x, y) in testpairs))
        end
    ) |> skipmissing |> all
end,
length(Jωs)

In [ ]:
using KernelDensity

In [ ]:
coefficients = Iterators.filter(x -> !isempty(x), values(Jωs) .|> skipmissing) |> collect .|> x -> abs.(x) |> mean;
histogram(coefficients, normalize=true,
    xlabel=L"Projection magnitude $\abs{\ip{\hat{\opr{P}}}{\opr{J}}}$",
    ylabel="Density",
    title=title(tim),
    legend=false);
coefficient_kde = kde(coefficients)
plot!(coefficient_kde.x, coefficient_kde.density)
vline!([1 / 2^tim.N])